In [3]:
#install latest Chrome driver to your machine https://sites.google.com/chromium.org/driver/
#install Beautiful Soup 
#install selenium

#imports
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import time
import json 


#base product URL
URL = "https://e-katanalotis.gov.gr/product/"
min_id = 12
max_id = 22

#creates a list [min_id...max_id-1]
productids = range(min_id, max_id)

#final JSON
results = {}
results['fetch_date']=int(time.time())
results['data']=[]

#get product information
for productid in productids:
    # if(productid==14 or productid==21 or productid==27 or  productid==30 or productid==38): continue
    print(productid)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    # executable_path param is where the Chrome driver is installed
    browser = webdriver.Chrome(options=options, executable_path='/usr/bin/chromedriver')
    browser.get(URL+str(productid))

    #get dates from price chart
    try:
        date_data = browser.execute_script('return Highcharts.charts[0].series[0].data.map(x => x.category)')
    except exceptions.JavascriptException as e:
        continue
    #append year to date labels
    date_data = [s +'/2022' for s in date_data]

    #get prices from price chart
    try:
        price_data = browser.execute_script('return Highcharts.charts[0].series[0].data.map(x => x.y)')
    except exceptions.JavascriptException as e:
        continue
    #find product name
    html = browser.page_source
    soup = BeautifulSoup(html, features="html5lib")

    #test html rendering
    # print(soup.prettify())

    try:
        pname = soup.find('p', attrs={'class':'product-name'}).get_text() 
    except AttributeError as e:
        continue
    browser.quit()

    #transform dates

    for i in range(len(date_data)-1):
        dparts = date_data[i].split("/")
        newd = '2022-'+dparts[1]+'-'+dparts[0]
        date_data[i]=newd
        
    #create a result object
    result = {}
    result['id']=productid
    result['name']=pname
    result['prices']=[]
    for i in range(len(date_data)-1):
        result['prices'].append({'date':date_data[i], 'price':price_data[i]})
    
    #append it to the list
    results['data'].append(result)

print('Done')

12


/tmp/ipykernel_2119/1772715251.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='/usr/bin/chromedriver')


13
14
Message: javascript error: Cannot read properties of undefined (reading 'series')
  (Session info: headless chrome=108.0.5359.94)
Stacktrace:
#0 0x55c306e472a3 <unknown>
#1 0x55c306c05f77 <unknown>
#2 0x55c306c099c8 <unknown>
#3 0x55c306c09792 <unknown>
#4 0x55c306c0a3fc <unknown>
#5 0x55c306c7af9e <unknown>
#6 0x55c306c62b32 <unknown>
#7 0x55c306c7a47c <unknown>
#8 0x55c306c62903 <unknown>
#9 0x55c306c35ece <unknown>
#10 0x55c306c36fde <unknown>
#11 0x55c306e9763e <unknown>
#12 0x55c306e9ab79 <unknown>
#13 0x55c306e7d89e <unknown>
#14 0x55c306e9ba83 <unknown>
#15 0x55c306e70505 <unknown>
#16 0x55c306ebcca8 <unknown>
#17 0x55c306ebce36 <unknown>
#18 0x55c306ed8333 <unknown>
#19 0x7fb5c2c56609 start_thread

Message: javascript error: Cannot read properties of undefined (reading 'series')
  (Session info: headless chrome=108.0.5359.94)
Stacktrace:
#0 0x55c306e472a3 <unknown>
#1 0x55c306c05f77 <unknown>
#2 0x55c306c099c8 <unknown>
#3 0x55c306c09792 <unknown>
#4 0x55c306c0a3fc <unkno

In [4]:
#inspect final JSON
json_object = json.dumps(results, indent = 2, ensure_ascii=False).encode('utf8')
print(json_object.decode())

{
  "fetch_date": 1670315996,
  "data": [
    {
      "id": 12,
      "name": "Stella Artois Μπύρα 330ml",
      "prices": [
        {
          "date": "2022-11-26",
          "price": 1.34
        },
        {
          "date": "2022-11-28",
          "price": 1.34
        },
        {
          "date": "2022-11-30",
          "price": 1.33
        },
        {
          "date": "2022-12-01",
          "price": 1.34
        },
        {
          "date": "2022-12-02",
          "price": 1.34
        }
      ]
    },
    {
      "id": 13,
      "name": "McCain Πατάτες Tradition Σακ 1κλ",
      "prices": [
        {
          "date": "2022-11-26",
          "price": 3.36
        },
        {
          "date": "2022-11-28",
          "price": 3.4
        },
        {
          "date": "2022-11-30",
          "price": 3.49
        },
        {
          "date": "2022-12-01",
          "price": 3.49
        },
        {
          "date": "2022-12-02",
          "price": 3.49
        }
   

In [5]:
#save final JSON to file
with open("output_"+str(productids[0])+"_"+str(productids[len(productids)-1])+"_"+str(results['fetch_date'])+".json", "w", encoding='utf-8') as outfile:
    json.dump(results, outfile, ensure_ascii=False ,indent=2)